In [1]:
import pandas as pd

# Load your spreadsheet
df = pd.read_csv('../output/options_output.csv')

# Convert 'order_created_at' to datetime
df['order_created_at'] = pd.to_datetime(df['order_created_at'])

# Calculate the cost for each order
df['cost'] = df['price'] * df['processed_quantity'] * 100

# Determine if an order is opening or closing
df['is_opening'] = df['opening_strategy'].notna()

# Calculate the impact on P&L for each trade
df['pl_impact'] = df.apply(lambda row: 
    row['cost'] if row['direction'] == 'credit' else -row['cost'], axis=1)

# Calculate realized and unrealized P&L
def calculate_position_pl(group):
    opening_trades = group[group['is_opening']]
    closing_trades = group[~group['is_opening']]
    
    opening_pl = opening_trades['pl_impact'].sum()
    closing_pl = closing_trades['pl_impact'].sum()
    
    realized_pl = opening_pl + closing_pl if len(closing_trades) > 0 else 0
    unrealized_pl = opening_pl if len(closing_trades) == 0 else 0
    
    return pd.Series({
        'realized_pl': realized_pl,
        'unrealized_pl': unrealized_pl,
        'is_open': len(closing_trades) == 0,
        'total_quantity': group['processed_quantity'].sum(),
        'avg_price': (group['price'] * group['processed_quantity']).sum() / group['processed_quantity'].sum(),
        'last_direction': group['direction'].iloc[-1]
    })

# Group by unique option identifiers and calculate P&L
result = df.groupby(['chain_symbol', 'expiration_date', 'strike_price', 'option_type']) \
           .apply(calculate_position_pl) \
           .reset_index()

total_realized_pl = result['realized_pl'].sum()
total_unrealized_pl = result['unrealized_pl'].sum()

print(f"Total Realized P&L: ${total_realized_pl:.2f}")
print(f"Total Unrealized P&L: ${total_unrealized_pl:.2f}")
print(f"Total P&L: ${(total_realized_pl + total_unrealized_pl):.2f}")

# Show all positions with additional details
print("\nAll Positions:")
print(result[['chain_symbol', 'expiration_date', 'strike_price', 'option_type', 
              'realized_pl', 'unrealized_pl', 'is_open', 'total_quantity', 'avg_price', 'last_direction']])

# Export results
result.to_csv('../output/processed_option_orders.csv', index=False)

Total Realized P&L: $9974.00
Total Unrealized P&L: $-8008.00
Total P&L: $1966.00

All Positions:
    chain_symbol expiration_date  strike_price option_type  realized_pl  \
0           AAPL      2019-05-03         202.5         put        786.0   
1           AAPL      2019-05-03         205.0         put       -580.0   
2           AAPL      2019-05-03         217.5        call          0.0   
3           AAPL      2019-05-10         217.5        call         54.0   
4           AAPL      2019-08-23         235.0        call          0.0   
..           ...             ...           ...         ...          ...   
658          VXX      2024-01-19          16.0        call         -7.0   
659          WDC      2019-05-10          47.0         put        -96.0   
660          WDC      2024-05-03          57.0         put          0.0   
661          WFC      2024-06-28          58.0        call         87.0   
662         YEXT      2019-05-17          22.5        call        -10.0   

  

/tmp/ipykernel_301237/3024452600.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calculate_position_pl) \
